<center><img src="https://github.com/DACSS-PreProcessing/Week_1_main/blob/main/pics/LogoSimple.png?raw=true" width="700"></center>


# Concatenating Data Frames in Python

Concatenating is an operation at the data frame level. It is an easy operation when all the data frames have the **same** column names, and in the same position  (vertical concatenation).

For this example, there is a webpage in **fragilestatesindex.org** where we can find several links to excel files. Let me get all the links:

In [1]:
import requests
from bs4 import BeautifulSoup
 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
url = "https://fragilestatesindex.org/excel/"
data = requests.get(url,headers=headers).text
soup = BeautifulSoup(data, 'html.parser')

allLinks=[]
for table in soup.find_all('table'):
    for a in table.find_all('a'):
        allLinks.append(a['href'].strip())
allLinks=set(allLinks)

In [2]:
allLinks

{'https://fragilestatesindex.org/wp-content/uploads/2018/04/fsi-2018.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/2019/04/fsi-2019.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/2020/05/fsi-2020.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/2021/05/fsi-2021.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/2022/07/fsi-2022-download.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/2023/06/FSI-2023-DOWNLOAD.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2006.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2007.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2008.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2009.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2010.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2011.xlsx',
 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2012.xlsx',
 'https://fragi

Now, I will create a list of data frames:

In [109]:
import pandas as pd

dfs=[] # a list
for link in allLinks:
    dfs.append(pd.read_excel(link,storage_options=headers))

We are not merging, but we should avoid duplicates after concatenating.

What columns do we have?

In [110]:
# saving column names
allColumnNames=[]
for df in dfs:
    allColumnNames.append(set(df.columns))# list of sets!

In [111]:
# details of common columns
commonColumns=set.intersection(*allColumnNames) # expanding list of sets (*)
commonColumns

{'C1: Security Apparatus',
 'C2: Factionalized Elites',
 'C3: Group Grievance',
 'Country',
 'E1: Economy',
 'E2: Economic Inequality',
 'E3: Human Flight and Brain Drain',
 'P1: State Legitimacy',
 'P2: Public Services',
 'P3: Human Rights',
 'Rank',
 'S1: Demographic Pressures',
 'S2: Refugees and IDPs',
 'Total',
 'X1: External Intervention',
 'Year'}

In [112]:
# all minus the common
set.union(*allColumnNames)-commonColumns

{'Change from Previous Year'}

All countries have the Country column. Let's see how they look:

In [113]:
allKeys=[] # list for contry names
for df in dfs:
    allKeys.append(set(df.Country))# list of sets!
#
commonKeys=set.intersection(*allKeys) 

# any weird pattern ?
set.union(*allKeys)-commonKeys

{'Antigua and Barbuda',
 'Armenia',
 'Armenia ',
 'Austria',
 'Austria ',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Belize',
 'Brunei Darussalam',
 'Cabo Verde',
 'Cambodia',
 'Cambodia ',
 'Canada',
 'Canada ',
 'Cape Verde',
 'Chile',
 'Chile ',
 'Comoros',
 'Congo Republic',
 "Cote d'Ivoire",
 'Croatia',
 'Croatia ',
 'Cyprus',
 'Cyprus ',
 'Czech Republic',
 'Czechia',
 "Côte d'Ivoire",
 'Djibouti',
 'Estonia',
 'Estonia ',
 'Eswatini',
 'Fiji',
 'Finland',
 'Finland ',
 'Georgia',
 'Georgia ',
 'Germany',
 'Germany ',
 'Ghana',
 'Ghana ',
 'Greece',
 'Greece ',
 'Grenada',
 'Guyana',
 'Iceland',
 'Ireland',
 'Ireland ',
 'Israel',
 'Israel and West Bank',
 'Italy',
 'Italy ',
 'Jamaica',
 'Jamaica ',
 'Japan',
 'Japan ',
 'Jordan',
 'Jordan ',
 'Kuwait',
 'Kuwait ',
 'Kyrgyz Republic',
 'Kyrgyzstan',
 'Lesotho',
 'Luxembourg',
 'Macedonia',
 'Macedonia ',
 'Madagascar',
 'Maldives',
 'Mali',
 'Mali ',
 'Malta',
 'Mauritius',
 'Mauritius ',
 'Micronesia',
 'Mongolia',
 'Mongolia ',
 '

We see that there are two simple problems affecting duplicates: the trailing spaces, and the accent symbol. Let's deal with that:

In [114]:
from unidecode import unidecode

for i in range(len(dfs)):
    dfs[i]['Country']=dfs[i].Country.str.strip().apply(unidecode)

Let's see what we have:

In [115]:
allKeys=[]
for df in dfs:
    allKeys.append(set(df.Country))# list of sets!

commonKeys=set.intersection(*allKeys)
set.union(*allKeys)-commonKeys

{'Antigua and Barbuda',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Belize',
 'Brunei Darussalam',
 'Cabo Verde',
 'Cape Verde',
 'Comoros',
 'Congo Republic',
 'Czech Republic',
 'Czechia',
 'Djibouti',
 'Eswatini',
 'Fiji',
 'Grenada',
 'Guyana',
 'Iceland',
 'Israel',
 'Israel and West Bank',
 'Kyrgyz Republic',
 'Kyrgyzstan',
 'Lesotho',
 'Luxembourg',
 'Macedonia',
 'Madagascar',
 'Maldives',
 'Malta',
 'Micronesia',
 'Montenegro',
 'North Macedonia',
 'Palestine',
 'Qatar',
 'Samoa',
 'Sao Tome and Principe',
 'Seychelles',
 'Slovak Republic',
 'Slovakia',
 'Solomon Islands',
 'South Sudan',
 'Suriname',
 'Swaziland',
 'Timor-Leste',
 'Trinidad and Tobago',
 'United Arab Emirates'}

The problem is more complicated now. There are countries that have been written in a different way. If we continue, we will create innecessary missing values if reshaped. Let's deal with this.

In [128]:
missfits=list(set.union(*allKeys)-commonKeys)

from thefuzz import process as fz

[(c,fz.extract(c,missfits,limit=2)) for c in sorted(missfits)]

[('Antigua and Barbuda', [('Antigua and Barbuda', 100), ('Barbados', 60)]),
 ('Bahamas', [('Bahamas', 100), ('Bahrain', 57)]),
 ('Bahrain', [('Bahrain', 100), ('Bahamas', 57)]),
 ('Barbados', [('Barbados', 100), ('Antigua and Barbuda', 60)]),
 ('Belize', [('Belize', 100), ('Congo Republic', 54)]),
 ('Brunei Darussalam', [('Brunei Darussalam', 100), ('Malta', 60)]),
 ('Cabo Verde', [('Cabo Verde', 100), ('Cape Verde', 80)]),
 ('Cape Verde', [('Cape Verde', 100), ('Cabo Verde', 80)]),
 ('Comoros', [('Comoros', 100), ('Congo Republic', 51)]),
 ('Congo Republic', [('Congo Republic', 100), ('Czech Republic', 71)]),
 ('Czech Republic', [('Czech Republic', 100), ('Czechia', 75)]),
 ('Czechia', [('Czechia', 100), ('Czech Republic', 75)]),
 ('Djibouti', [('Djibouti', 100), ('Fiji', 51)]),
 ('Eswatini', [('Eswatini', 100), ('Palestine', 59)]),
 ('Fiji', [('Fiji', 100), ('Djibouti', 51)]),
 ('Grenada', [('Grenada', 100), ('Trinidad and Tobago', 51)]),
 ('Guyana', [('Guyana', 100), ('Antigua and B

Only the second best is useful. Then:

In [129]:
[(c,fz.extract(c,missfits,limit=2)[1]) for c in sorted(missfits) if (c,fz.extract(c,missfits,limit=2)[1])[1][1]>=75]

[('Cabo Verde', ('Cape Verde', 80)),
 ('Cape Verde', ('Cabo Verde', 80)),
 ('Czech Republic', ('Czechia', 75)),
 ('Czechia', ('Czech Republic', 75)),
 ('Israel', ('Israel and West Bank', 90)),
 ('Israel and West Bank', ('Israel', 90)),
 ('Macedonia', ('North Macedonia', 90)),
 ('North Macedonia', ('Macedonia', 90)),
 ('Slovak Republic', ('Slovakia', 77)),
 ('Slovakia', ('Slovak Republic', 77))]

We should prepare a dictionary of changes, some will be input manually:

In [130]:
theFits=[(c,fz.extract(c,missfits,limit=2)[1]) for c in sorted(missfits) if (c,fz.extract(c,missfits,limit=2)[1])[1][1]>=75]
allChanges={fit[0]:fit[1][0] for fit in theFits[0:-1:2]}

allChanges.update({'Kyrgyzstan':'Kyrgyz Republic'})
allChanges.update({'Swaziland':'Eswatini'})
allChanges

{'Cabo Verde': 'Cape Verde',
 'Czech Republic': 'Czechia',
 'Israel': 'Israel and West Bank',
 'Macedonia': 'North Macedonia',
 'Slovak Republic': 'Slovakia',
 'Kyrgyzstan': 'Kyrgyz Republic',
 'Swaziland': 'Eswatini'}

In [78]:
for i in range(len(dfs)):
    dfs[i].replace({'Country':allChanges}, inplace=True)

In [79]:
allKeys=[]
for df in dfs:
    allKeys.append(set(df.Country))# list of sets!

commonKeys=set.intersection(*allKeys)
set.union(*allKeys)-commonKeys

{'Antigua and Barbuda',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Belize',
 'Brunei Darussalam',
 'Cape Verde',
 'Comoros',
 'Congo Republic',
 'Djibouti',
 'Eswatini',
 'Fiji',
 'Grenada',
 'Guyana',
 'Iceland',
 'Lesotho',
 'Luxembourg',
 'Madagascar',
 'Maldives',
 'Malta',
 'Micronesia',
 'Montenegro',
 'Palestine',
 'Qatar',
 'Samoa',
 'Sao Tome and Principe',
 'Seychelles',
 'Solomon Islands',
 'South Sudan',
 'Suriname',
 'Timor-Leste',
 'Trinidad and Tobago',
 'United Arab Emirates'}

Now, you can use the list to concatenate:

In [80]:
allDFs=pd.concat(objs=dfs, # DFs as a list
                 axis=0, # one DF on top of the other
                 ignore_index=True, #very important
                 copy=False)

allDFs.columns

Index(['Country', 'Year', 'Rank', 'Total', 'C1: Security Apparatus',
       'C2: Factionalized Elites', 'C3: Group Grievance', 'E1: Economy',
       'E2: Economic Inequality', 'E3: Human Flight and Brain Drain',
       'P1: State Legitimacy', 'P2: Public Services', 'P3: Human Rights',
       'S1: Demographic Pressures', 'S2: Refugees and IDPs',
       'X1: External Intervention', 'Change from Previous Year'],
      dtype='object')

Some basic cleaning in column names:

In [81]:
allDFs.columns=allDFs.columns.str.replace(r':\s|\s','_',regex=True)

allDFs

,Country,Year,Rank,Total,C1_Security_Apparatus,C2_Factionalized_Elites,C3_Group_Grievance,E1_Economy,E2_Economic_Inequality,E3_Human_Flight_and_Brain_Drain,P1_State_Legitimacy,P2_Public_Services,P3_Human_Rights,S1_Demographic_Pressures,S2_Refugees_and_IDPs,X1_External_Intervention,Change_from_Previous_Year
0,Yemen,2020-01-01 00:00:00,1st,112.438694,9.700000,10.0,9.69887,9.400000,7.800000,7.000000,9.889823,9.500000,9.950000,9.800000,9.7,10.000000,-1.061306
1,Somalia,2020-01-01 00:00:00,2nd,110.888959,9.811328,10.0,8.60000,9.100000,9.367151,8.900000,8.888107,9.100000,9.000000,9.981087,9.1,9.041286,-1.311041
2,South Sudan,2020-01-01 00:00:00,3rd,110.752190,9.400000,9.7,9.10000,9.500000,9.200000,6.800000,9.944415,9.500000,9.000000,9.450357,9.7,9.457419,-1.447810
3,Syria,2020-01-01 00:00:00,4th,110.749697,9.900000,9.9,10.00000,8.686367,7.200000,8.413343,9.950000,9.100000,10.000000,7.600000,10.0,9.999986,-0.850303
4,Congo Democratic Republic,2020-01-01 00:00:00,5th,109.394621,8.500000,9.8,9.70000,8.000000,8.619842,6.900000,9.660971,9.464911,9.548897,9.800000,10.0,9.400000,-0.805379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,Sweden,2017-01-01 00:00:00,174th,22.100000,2.100000,1.8,1.50000,1.800000,1.800000,1.500000,1.000000,1.100000,1.100000,2.000000,5.3,1.100000,NaN
3166,Denmark,2017-01-01 00:00:00,175th,21.500000,1.700000,1.4,4.40000,2.000000,1.600000,1.900000,0.900000,1.000000,1.500000,2.000000,2.1,1.000000,NaN
3167,Switzerland,2017-01-01 00:00:00,176th,21.100000,1.100000,1.0,3.60000,2.300000,2.200000,2.100000,1.000000,1.000000,1.800000,1.500000,2.5,1.000000,NaN
3168,Norway,2017-01-01 00:00:00,177th,20.500000,2.000000,1.1,3.60000,2.200000,1.500000,1.600000,0.800000,1.000000,1.100000,1.500000,2.8,1.300000,NaN


In [82]:
allDFs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3170 entries, 0 to 3169
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          3170 non-null   object 
 1   Year                             3170 non-null   object 
 2   Rank                             3170 non-null   object 
 3   Total                            3170 non-null   float64
 4   C1_Security_Apparatus            3170 non-null   float64
 5   C2_Factionalized_Elites          3170 non-null   float64
 6   C3_Group_Grievance               3170 non-null   float64
 7   E1_Economy                       3170 non-null   float64
 8   E2_Economic_Inequality           3170 non-null   float64
 9   E3_Human_Flight_and_Brain_Drain  3170 non-null   float64
 10  P1_State_Legitimacy              3170 non-null   float64
 11  P2_Public_Services               3170 non-null   float64
 12  P3_Human_Rights     

We should drop the last column:

In [83]:
allDFs.drop(columns=['Change_from_Previous_Year'], inplace=True)

Pay attention to years:

In [84]:
allDFs.Year.value_counts()

Year
2021                   179
2022-01-01 00:00:00    179
2023                   179
2020-01-01 00:00:00    178
2019-01-01 00:00:00    178
2015-01-01 00:00:00    178
2014-01-01 00:00:00    178
2018-01-01 00:00:00    178
2012-01-01 00:00:00    178
2013-01-01 00:00:00    178
2016-01-01 00:00:00    178
2017-01-01 00:00:00    178
2009-01-01 00:00:00    177
2011-01-01 00:00:00    177
2010-01-01 00:00:00    177
2008-01-01 00:00:00    177
2007-01-01 00:00:00    177
2006-01-01 00:00:00    146
Name: count, dtype: int64

These years need formatting:

In [85]:
allDFs['Year']=[y if type(y)==int else y.year for y in allDFs.Year  ]

In [86]:
allDFs.Year.value_counts()

Year
2021    179
2022    179
2023    179
2020    178
2019    178
2015    178
2014    178
2018    178
2012    178
2013    178
2016    178
2017    178
2009    177
2011    177
2010    177
2008    177
2007    177
2006    146
Name: count, dtype: int64

In [97]:
allDFs[allDFs.isnull().any(axis=1)]

,Country,Year,Rank,Total,C1_Security_Apparatus,C2_Factionalized_Elites,C3_Group_Grievance,E1_Economy,E2_Economic_Inequality,E3_Human_Flight_and_Brain_Drain,P1_State_Legitimacy,P2_Public_Services,P3_Human_Rights,S1_Demographic_Pressures,S2_Refugees_and_IDPs,X1_External_Intervention


Let's save the DF:

In [ ]:
allDFs.to_csv('allDFs.csv',index=False)